In [2]:
import numpy as np
import pandas as pd
import gc,random,os
from sklearn.preprocessing import PowerTransformer,MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.mixture import BayesianGaussianMixture
from sklearn import metrics
import lightgbm as lgb

ModuleNotFoundError: No module named 'numpy'

In [3]:
!pip list

Package             Version             
------------------- --------------------
asttokens           2.0.5               
backcall            0.2.0               
certifi             2019.11.28          
chardet             3.0.4               
dbus-python         1.2.16              
debugpy             1.6.2               
decorator           5.1.1               
entrypoints         0.4                 
executing           0.8.3               
idna                2.8                 
ipykernel           6.15.1              
ipython             8.4.0               
jedi                0.18.1              
jupyter-client      7.3.4               
jupyter-core        4.11.1              
matplotlib-inline   0.1.3               
nest-asyncio        1.5.5               
packaging           21.3                
parso               0.8.3               
pexpect             4.8.0               
pickleshare         0.7.5               
pip                 20.0.2              
prompt-toolkit  